In [1]:
# No more COMPLETE layers
# PARTIALLY CONNECTED -> some amount of learning
# Complex network of partially connected layers 
# FLATTEN + Dense(ReLU) + Dense (Softwax)
# colored/multi-dim images to something that can be accepted by flattening layer



In [2]:
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np
print(tf.__version__)

TensorFlow 1.x selected.
1.15.2


In [3]:
# VGG = SOme feature_generator_thru_conv X repeat_factor + image_classification
# Conv -> BatchNormalize -> Conv -> BatchNormalize.....  
# no neg numbers
# Conv -> ReLU -> BN -> Conv -> Relu -> BN ..... 
# Compressing the image->
# First step-> learn small filters on large image
# LAST step-> learn large filters on small image
# Conv -> ReLU -> BN-> MP -> Conv -> Relu -> BN -> MP ..... 
# Regularize the network for better output
# DROPOUT layer 
# RandomVGG=((Conv2D->Relu-> BN)+MP+Dropout)XN + FLATTEN + Dense(ReLU) + Dense (Softwax)

In [5]:
from keras import backend
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential

In [6]:
#N= 5
HP_block1_conv_dim = 32
HP_block2_conv_dim = 64
HP_block3_conv_dim = 128
HP_block4_conv_dim = 256
HP_block5_dense_dim = 1024
HP_small_pattern = (3,3) # UNCOMPRESSED or 1-2 compression IMAGES
HP_large_pattern = (2,2) # 4 times compressed images from previous MP layers!!!
HP_dropout_little =0.25
HP_dropout_big = 0.50
# HP_epochs, batch_size-> are now problems of the developer USING this model. 




In [10]:
class RacoonVGG:
  @staticmethod
  def build(height, width, depth, classes):
    # assume that we are on TF, but if something else is detected, switch the dimension
    input_shape = (height, width, depth)
    channel_dim = -1 # last position 
    if backend.image_data_format() == 'channels_first':
      input_shape = (depth, height, width)
      channel_dim = 1
    model = Sequential()
    
    # BLOCK1
    model.add(Conv2D(HP_block1_conv_dim,HP_small_pattern, padding='same',
                     input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(MaxPooling2D(pool_size=HP_small_pattern))
    model.add(Dropout(HP_dropout_little))

    # COMPLEX BLOCK 2
    model.add(Conv2D(HP_block2_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(Conv2D(HP_block2_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(MaxPooling2D(pool_size=HP_large_pattern))
    model.add(Dropout(HP_dropout_little))
    
    # COMPLEX BLOCK 3
    model.add(Conv2D(HP_block3_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(Conv2D(HP_block3_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(MaxPooling2D(pool_size=HP_large_pattern))
    model.add(Dropout(HP_dropout_little))

    # COMPLEX BLOCK 4
    model.add(Conv2D(HP_block4_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(Conv2D(HP_block4_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(MaxPooling2D(pool_size=HP_large_pattern))
    model.add(Dropout(HP_dropout_little))

    # BLOCK 5- Image Classification (OBJECT)
    model.add(Flatten())
    model.add(Dense(HP_block5_dense_dim))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(HP_dropout_big))
    model.add(Dense(classes))
    model.add(Activation('softmax'))
    


    # TF-> (h,w,d)
    # Theano-> (d,h,w)
    # GENERIC PROGRAM-> support BOTH the methods!!! 
    # if using KERAS-> because KERAS is just an API that is front facing an unknown
    # backend!